In [1]:
import numpy as np
import fitsio as fi
import treecorr
import matplotlib.pyplot as plt
plt.style.use('y1a1')
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline


In [2]:
# Read in the catalogues # Read  
base = '/Users/hattifattener/Documents/ias/mbii/ia_modelling_public/catalogues/fiducial/'
data = fi.FITS(base+'mbii-ndm1000-nst300-snapshot85.fits')[-1].read()

### TreeCorr Approx.

In [3]:
# First TreeCorr
# Main calculation
cat = treecorr.Catalog(x=data['x'], y=data['y'], z=data['z'])
cc = treecorr.NNCorrelation(min_sep=0.1, max_sep=33., nbins=21)

cc.process(cat,cat)

In [4]:
# Also do the randoms
rr = treecorr.NNCorrelation(min_sep=0.1, max_sep=33., nbins=21)
rn = treecorr.NNCorrelation(min_sep=0.1, max_sep=33., nbins=21)
nr = treecorr.NNCorrelation(min_sep=0.1, max_sep=33., nbins=21)

def randoms(cat1, period=100):
    # Initialise randoms
    np.random.seed(9000)
    rcat = treecorr.Catalog(x=np.random.rand(cat.ntot)*period, y=np.random.rand(cat.ntot)*period, z=np.random.rand(cat.ntot)*period)
    return rcat

rcat = randoms(cat, period=100)

print('Processing randoms',)
print('RR',)
rr.process(rcat,rcat)
print('DR',)
nr.process(cat,rcat)
print('RD',)
rn.process(rcat,cat)

# Finish off
xi, var = cc.calculateXi(rr, dr=nr, rd=rn)

Processing randoms
RR
DR
RD


### TreeCorr Exact.

In [ ]:
cc0 = treecorr.NNCorrelation(min_sep=0.1, max_sep=33., nbins=21, bin_slop=0.0)
cc0.process(cat,cat)

rr = treecorr.NNCorrelation(min_sep=0.1, max_sep=33., nbins=21, bin_slop=0.0)
rn = treecorr.NNCorrelation(min_sep=0.1, max_sep=33., nbins=21, bin_slop=0.0)
nr = treecorr.NNCorrelation(min_sep=0.1, max_sep=33., nbins=21, bin_slop=0.0)

print('Processing randoms',)
print('RR',)
rr.process(rcat,rcat)
print('DR',)
nr.process(cat,rcat)
print('RD',)
rn.process(rcat,cat)

# Finish off
xi0, var0 = cc0.calculateXi(rr, dr=nr, rd=rn)

Processing randoms
RR
DR


### Halotools Exact.

In [ ]:
from halotools.mock_observables.two_point_clustering import tpcf 

pvec = np.vstack((data['x'], data['y'], data['z'])).T
rvec = np.vstack((rcat.x, rcat.y, rcat.z)).T
rbins = np.logspace(np.log10(0.1), np.log10(33.), 22 )

gg = tpcf(pvec, rbins, randoms=rvec, num_threads=1, estimator='Landy-Szalay') 

In [ ]:
diff = (xi-gg)/gg
diff0 = (xi0-gg)/gg
x = np.sqrt(rbins[1:]*rbins[:-1])


plt.plot(x, diff, '+', color='k', label='slop$=$0.1')
plt.plot(x, diff0, 'x', color='purple', label='slop$=$0.0')
plt.axhline(0,color='k',ls=':')
plt.xscale('log')
plt.ylim(-0.008,0.008)
plt.legend(loc='lower left')
plt.ylabel('Frac. Diff.')
plt.xlabel(r'Comoving Separation $r$ / $h^{-1}$Mpc')
plt.show()
print(100*diff)